In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\CJ\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv("bike-sharing-demand/train.csv", parse_dates=["datetime"])
test = pd.read_csv("bike-sharing-demand/test.csv", parse_dates=["datetime"])

In [3]:
train['year'] = train['datetime'].dt.year
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['minute'] = train['datetime'].dt.minute
train['second'] = train['datetime'].dt.second
train['dayofweek'] = train['datetime'].dt.dayofweek

In [4]:
test['year'] = test['datetime'].dt.year
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['minute'] = test['datetime'].dt.minute
test['second'] = test['datetime'].dt.second
test['dayofweek'] = test['datetime'].dt.dayofweek

In [5]:
train.shape, test.shape

((10886, 19), (6493, 16))

# Feature Engineering

## `windspeed`

### 방법 1 : 평균값 이용

In [6]:
# 풍속(windspeed)의 0값을 풍속의 평균값으로 대체
# for dataset in train_test:
#     dataset.windspeed.replace(0,dataset.windspeed.mean(),inplace=True)
# 평균으로 대체하는 것은 좋은 방법이 아님

### 방법 2: 랜덤포레스트 이용

#### windspeed 예측하기

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek'],
      dtype='object')

In [9]:
def predictWindSpeed(data):
    #data의 windspeed열 값 0을 랜덤포레스트 기반 예측값으로 대체
    #풍속 0, not0 으로 분류
    dataWind0 = data.copy().loc[data.windspeed==0]
    dataWindNot0 = data.copy().loc[data.windspeed!=0]
    
    #입력데이터 -> 랜덤포레스트 모델 -> 출력데이터(windspeed)
    #풍속을 예측하는데 사용될 변수(입력)을 선택 -> 정석은 상관관계분석을 통해 feature를 선정해야 함
    wCol = ['season', 'weather', 'temp', 'atemp', 'humidity', 'year', 'month']
    
    #회귀모델(Linear Regression)
    # 풍속예측함수 = w1*season + w2*weather + ... + w7*month + b
    
    #출력데이터 타입이 str이어야 함
    dataWindNot0['windspeed'] = dataWindNot0['windspeed'].astype('str')
    
    #랜덤포레스트 분류기
    rfModelWind = RandomForestClassifier()
    
    #모델링(학습데이터) -> windspeed가 0이 아닌 데이터들로 훈련
    #windNot0 데이터로 학습(fit)을 시켜서 모델을 만든 후, wind0데이터를 모델에 입력하면 예상되는 풍속이 출력
    rfModelWind.fit(dataWindNot0[wCol], dataWindNot0.windspeed)
    
    #예측 : windspeed가 0인 데이터를 모델에 넣어 windspeed를 예측한다.
    dataWind0.windspeed = rfModelWind.predict(dataWind0[wCol])
    
    #데이터를 도로 합치기
    data = dataWindNot0.append(dataWind0)
    data.windspeed = data.windspeed.astype('float')
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    
    return data

In [10]:
train = predictWindSpeed(train)
#test = predictWindSpeed(test)

In [11]:
train.windspeed.describe()

count    10886.000000
mean        14.005211
std          7.052328
min          6.003200
25%          8.998100
50%         12.998000
75%         19.001200
max         56.996900
Name: windspeed, dtype: float64

In [12]:
#연속형 변수 -> 범주형 변수
category_fn = ['season', 'holiday', 'workingday', 'weather', 'year', 'month', 'hour', 'dayofweek']

for v in category_fn:
    train[v] = train[v].astype('category')
    test[v] = test[v].astype('category')

In [13]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  category      
 2   holiday     10886 non-null  category      
 3   workingday  10886 non-null  category      
 4   weather     10886 non-null  category      
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  category      
 13  month       10886 non-null  category      
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  category      
 16  minute      10886 non-

# Feature Selection

In [14]:
fn = ['season', 'holiday', 'workingday', 'weather', 'year', 'hour', 'dayofweek', 
      'temp', 'atemp', 'humidity', 'windspeed']

In [15]:
xTrain = train[fn]
yTrain = train['count']

xTest = test[fn]

# 모델링 (랜덤포레스트 회귀모델)

성능 평가 : k-fold 교차검증

In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=2020)

In [18]:
model.fit(xTrain, yTrain)

RandomForestRegressor(n_jobs=-1, random_state=2020)

# 모델 성능 평가

### RMSLE : 평가 지표(Kaggle의 해당 competition 평가기준)

\\(\sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 }\\)

In [19]:
from sklearn.metrics import make_scorer

In [20]:
def rmsle(pv, av): #예측값, 실제값 비교하여 정확도 계산
    #넘파이 배열로 변환
    pv = np.array(pv)
    av = np.array(av)
    
    #예측값과 실제값에 1을 더하고 로그를 씌운다
    log_predict = np.log(pv+1)
    log_actual = np.log(av+1)
    
    res = log_predict-log_actual
    res = np.square(res)
    
    mean_res = res.mean()
    score = np.sqrt(mean_res)
    
    return score

In [21]:
rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

---
[참고]  

__make_scorer()__:
   - 자신만의 측정 지표를 정의 할 때 사용.  
   - **두 개의 매개변수**를 가진 함수를 정의 한 후 `make_scorer`함수를 사용해 스코어객체를 만듦.  
   - 높은 점수와 낮은 점수 중에 `great_is_better` 매개변수를 사용해 바람직한 것을 지정
    
---

### k-fold 교차검증

In [22]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [23]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42) 
#데이터의 순서를 섞어서 나누기(특징별로 sorting되어있을 경우를 방지)

In [24]:
score = cross_val_score(model, xTrain, yTrain, cv=kfold, scoring=rmsle_scorer)
"""
xTrain이 model에 들어가서 나온 예측결과 : pv
yTrain : av
=> 이 두 변수를 rmsle함수에 전달
=> score 리턴
"""
score

array([0.29332381, 0.32497393, 0.34683206, 0.33036135, 0.32799477,
       0.32663021, 0.35495788, 0.332829  , 0.32412196, 0.35536157])

# 예측

In [25]:
pred = model.predict(xTest)
pred #예측결과

array([ 11.73      ,   4.6       ,   4.15      , ..., 100.725     ,
       106.51666667,  47.46      ])

In [26]:
bikesubmit = pd.read_csv("bike-sharing-demand/sampleSubmission.csv")
bikesubmit['count'] = pred
bikesubmit

,datetime,count
0,2011-01-20 00:00:00,11.730000
1,2011-01-20 01:00:00,4.600000
2,2011-01-20 02:00:00,4.150000
3,2011-01-20 03:00:00,3.340000
4,2011-01-20 04:00:00,3.060000
...,...,...
6488,2012-12-31 19:00:00,204.030000
6489,2012-12-31 20:00:00,162.780000
6490,2012-12-31 21:00:00,100.725000
6491,2012-12-31 22:00:00,106.516667


In [27]:
bikesubmit.to_csv("mysubmit_bike.csv", index=False)